In [30]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import pickle

import tensorflow as tf
from keras.applications.resnet import *
from keras.preprocessing.image_dataset import *
from keras.preprocessing import image
from keras.models import Model

In [18]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

for f in train_path:
    img = image.load_img('./train/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_i.append(x)
for f in test_path:
    img = image.load_img('./test/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_i.append(x)

In [31]:
base_model = ResNet152(weights = 'imagenet', include_top = True)
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()
#model = VGG19(weights='imagenet', include_top=False)

242909184/242900224 [==============================] - 44s 0us/step
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [33]:
train_preds = []
test_preds = []

for i in tqdm(train_i):
    pred = model.predict(i)
    train_preds.append(pred)

for i in tqdm(test_i):
    pred = model.predict(i)
    test_preds.append(pred)

100%|██████████| 1200/1200 [08:00<00:00,  2.50it/s]


In [34]:
f = open(f'./test_resnet152.pckl','wb')
pickle.dump(test_preds,f)
f.close()

f = open(f'./train_resnet152.pckl','wb')
pickle.dump(train_preds,f)
f.close()

In [35]:
best_train = []
for test in tqdm(test_preds):
    match = [np.sum((train-test)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)[:3]
    row = train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=['3','2','1'])
df.to_csv('resnet152_matches.csv',index=False)

100%|██████████| 1200/1200 [02:58<00:00,  6.73it/s]


In [36]:
out=[]
for i in range(len(df)):
    coor = [train_xy[train_xy['id']== df[label][i]] for label in ['1','2','3']]
    result = pd.concat(coor)
    out.append([test_path[i], np.mean(result['x']), np.mean(result['y'])])
    
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet152_top3_out.csv',index=False)